In [1]:
import pandas as pd
# df1 = pd.read_excel("raspagem_limpa_brasil_SP_rev.01.xlsx")
# df2 = pd.read_excel("raspagem_limpa_brasil_teste.xlsx")
# df = pd.concat([df1, df2], ignore_index=True)

In [19]:
df_base = pd.read_excel("raspagem_limpa_brasil_SP_rev.01.xlsx")
df_campanha = pd.read_excel("raspagem_limpa_brasil_teste.xlsx")
df_base['Campanha'] = df_campanha['Campanha']
df_sp = df_base[df_base['Cidade e Estado'].str.contains("São Paulo", case=False, na=False)].copy()
df_sp['Cidade e Estado'] = df_sp['Cidade e Estado'].str.replace(r'São Paulo\s*/\s*', '', regex=True).str.strip()
palavras_chave = ['Mutirão', 'MUTIRÃO', 'Multirão', 'mutirão', 'Limpeza', 'LIMPEZA', 'limpeza']
def separacao(titulo):
    if not isinstance(titulo, str):
        return False
    return any(palavra in titulo.split(" ") for palavra in palavras_chave)
df_sp['É Mutirão'] = df_sp['Título'].apply(separacao)
df_mutirao = df_sp[df_sp['É Mutirão']].copy()
df_eventos = df_sp[~df_sp['É Mutirão']].copy()
df_mutirao['Origem Base Dados'] = 'BD_Raspagem'
df_mutirao['Campanha'] = df_mutirao['Campanha'].fillna('').str.title()
df_mutirao['Título'] = df_mutirao['Título'].str.title()
df_mutirao['Tipo Campanha'] = 'Mutirão'
df_eventos['Origem Base Dados'] = 'BD_Raspagem'
df_eventos['Campanha'] = df_eventos['Campanha'].fillna('').str.title()
df_eventos['Título'] = df_eventos['Título'].str.title()
df_eventos['Tipo Campanha'] = 'Educação'
ordem_coluna = ['Data e Hora', 'Cidade e Estado', 'Campanha', 'Título', 'Tipo Campanha', 'Origem Base Dados']
df_mutirao = df_mutirao[ordem_coluna]
df_eventos = df_eventos[ordem_coluna]
def converter_data_br(df, nome_coluna):
    df['Data Original'] = df[nome_coluna]
    df[nome_coluna] = pd.to_datetime(df[nome_coluna], format='%d/%m/%Y %H:%M', errors='coerce')
    if df[nome_coluna].isna().any():
        df[nome_coluna] = pd.to_datetime(df['Data Original'], errors='coerce')
    erros = df[df[nome_coluna].isna() & df['Data Original'].notna()]
    if not erros.empty:
        print("Datas com erro de conversão:")
        print(erros[['Data Original', 'Título']])
    df[nome_coluna] = df[nome_coluna].dt.strftime('%d/%m/%Y')
    df.drop(columns=['Data Original'], inplace=True)
converter_data_br(df_mutirao, 'Data e Hora')
converter_data_br(df_eventos, 'Data e Hora')
df_mutirao.to_excel('planilha_mutirao.xlsx', index=False)
df_eventos.to_excel('planilha_educacao.xlsx', index=False)